# Watson ML for Developers

## Import, Clean, and Analyze Data
We are going to use PixieDust to load and visualize our data.

In [ ]:
import pixiedust

In [ ]:
df = pixiedust.sampleData("https://raw.githubusercontent.com/markwatsonatx/watson-ml-for-developers/master/data/house-prices.csv")

In [ ]:
display(df)

## Build a Machine Learning Model with Spark ML 

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler

In [ ]:
assembler = VectorAssembler(inputCols=['SquareFeet','Bedrooms'],outputCol="features")
lr = LinearRegression(labelCol='Price', featuresCol='features')
pipeline = Pipeline(stages=[assembler, lr])
model = pipeline.fit(df)

### Test the Model

In [ ]:
def get_prediction(square_feet, num_bedrooms):
    request_df = spark.createDataFrame([(square_feet, num_bedrooms)], ['SquareFeet','Bedrooms'])
    response_df = model.transform(request_df)
    return response_df

In [ ]:
response = get_prediction(2400, 4)
response.show()

# Deploy Model to Watson ML

In [ ]:
import json
import requests
import urllib3

### Fill in Your Watson ML Credentials

In [ ]:
service_path = 'https://ibm-watson-ml.mybluemix.net'
username = 'YOUR_WML_USER_NAME'
password = 'YOUR_WML_PASSWORD'
instance_id = 'YOUR_WML_INSTANCE_ID'
model_name = 'House Prices Model'
deployment_name = 'House Prices Deployment'

In [ ]:
# The code was removed by DSX for sharing.

### Initialize Watson ML Repository Libraries

In [ ]:
from repository.mlrepositoryclient import MLRepositoryClient
from repository.mlrepositoryartifact import MLRepositoryArtifact
ml_repository_client = MLRepositoryClient(service_path)
ml_repository_client.authorize(username, password)

### Save Model to Watson ML Repository

In [ ]:
pipeline_artifact = MLRepositoryArtifact(pipeline, name="pipeline")
model_artifact = MLRepositoryArtifact(model, training_data=df, name=model_name, pipeline_artifact=pipeline_artifact)
saved_model = ml_repository_client.models.save(model_artifact)
model_id = saved_model.uid

In [ ]:
headers = urllib3.util.make_headers(basic_auth='{}:{}'.format(username, password))
url = '{}/v3/identity/token'.format(service_path)
response = requests.get(url, headers=headers)
ml_token = 'Bearer ' + json.loads(response.text).get('token')

### Create a Deployment

In [ ]:
deployment_url = service_path + "/v3/wml_instances/" + instance_id + "/published_models/" + model_id + "/deployments/"
deployment_header = {'Content-Type': 'application/json', 'Authorization': ml_token}
deployment_payload = {"type": "online", "name": deployment_name}
deployment_response = requests.post(deployment_url, json=deployment_payload, headers=deployment_header)
deployment_response.text
scoring_url = json.loads(deployment_response.text).get('entity').get('scoring_url')
scoring_url

### Test the Deployment

In [ ]:
def get_prediction_from_watson_ml(square_feet, num_bedrooms):
    scoring_header = {'Content-Type': 'application/json', 'Authorization': ml_token}
    scoring_payload = {'fields': ['SquareFeet','Bedrooms'], 'values': [[square_feet, num_bedrooms]]}
    scoring_response = requests.post(scoring_url, json=scoring_payload, headers=scoring_header)
    return scoring_response.text

In [ ]:
response = get_prediction_from_watson_ml(2400, 4)
print response